# Hyperparameter Tuning using HyperDrive

In [ ]:
# Imports

import os
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.dataset import Dataset

## Dataset

In [ ]:
# Worspace and experiment

ws = Workspace.from_config()
experiment_name = 'hyperdrive-experiment'
#project_folder = './hyperdrive-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
# Create compute cluster

compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
# Dataset

found = False
key = "Heart Failure Dataset"
description_text = "Heart Failure Dataset for Udacity Project 3"

if key in ws.datasets.keys():
        found = True
        dataset = ws.datasets[key]

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/peppegili/3_Capstone_Project_ML_Engineer/blob/master/data/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

In [ ]:
# Specify a policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Specify parameter sampler
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice(25, 50, 100, 150)
    }
)

# if "training" not in os.listdir():
#     os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
estimator = SKLearn(source_directory=".", compute_target=compute_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=20,
                                         max_concurrent_runs=4)

In [ ]:
# Submit the experiment
hdr = exp.submit(hyperdrive_run_config)

## Run Details

In [ ]:
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

assert(hdr.get_status() == "Completed")

## Best Model

In [ ]:
best_run_hdr = hdr.get_best_run_by_primary_metric()
best_run_metrics_hdr = best_run_hdr.get_metrics()
best_params_hdr = best_run_hdr.get_details()['runDefinition']['arguments']

print("\n")
print('Best run ID: ', best_run_hdr.id)
print('Best run Accuracy: ', best_run_metrics_hdr['Accuracy'])
print('Metrics: ', best_run_metrics_hdr)
print("\n")

print(best_run_hdr.get_file_names())

In [ ]:
# Save the best model
joblib.dump(best_params_hdr, filename='outputs/best_model_hyperdrive.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service